# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [25]:
from __future__ import division
import turicreate
import math
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [3]:
products = turicreate.SFrame('amazon_baby.sframe/')

Now, let us see a preview of what the dataset looks like.

In [5]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [6]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [34]:
import string 
def remove_punctuation(text):
    translator = text.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = turicreate.text_analytics.count_words(review_without_punctuation)

products[0]

{'name': 'Planetwise Wipe Pouch',
 'review': 'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.',
 'rating': 5.0,
 'word_count': {'recommend': 1.0,
  'moist': 1.0,
  'osocozy': 1.0,
  'keps': 1.0,
  'leak': 1.0,
  'holder': 1.0,
  'was': 1.0,
  'now': 1.0,
  'wipe': 1.0,
  'my': 2.0,
  'disappointed': 1.0,
  'wise': 1.0,
  'and': 3.0,
  'planet': 1.0,
  'it': 3.0,
  'love': 1.0,
  'early': 1.0,
  'not': 2.0,
  'highly': 1.0,
  'wipes': 1.0,
  'bags': 1.0,
  'came': 1.0,
  'does': 1.0,
  'i': 1.0},
 'sentiment': 1}

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [35]:
print(products[269]['word_count'])

{'baby': 1.0, 'has': 1.0, 'anyone': 1.0, 'book': 1.0, 'sears': 1.0, 'saver': 1.0, 'use': 1.0, 'no': 1.0, 'older': 1.0, 'now': 1.0, 'dr': 1.0, 'put': 1.0, 'slipped': 1.0, 'life': 1.0, 'just': 1.0, 'her': 2.0, 'very': 1.0, 'bring': 1.0, 'great': 2.0, 'up': 1.0, 'way': 1.0, 'sit': 2.0, 'who': 1.0, 'at': 1.0, 'longer': 1.0, 'down': 3.0, 'lie': 2.0, 'wanted': 1.0, 'then': 1.0, 'far': 1.0, 'had': 1.0, 'spread': 1.0, 'reviews': 1.0, 'they': 1.0, 'but': 3.0, 'recomend': 1.0, 'because': 1.0, 'does': 1.0, 'having': 1.0, 'one': 1.0, 'bought': 1.0, 'get': 2.0, 'easy': 1.0, 'quite': 1.0, 'move': 1.0, 'and': 4.0, 'too': 1.0, 'daughter': 1.0, 'times': 1.0, 'am': 1.0, 'when': 2.0, 'a': 2.0, 'could': 2.0, 'we': 3.0, 'only': 2.0, '6quot4': 1.0, 'do': 1.0, 'or': 1.0, 'husbad': 1.0, 'such': 1.0, 'switching': 1.0, 'others': 1.0, 'fit': 1.0, 'not': 4.0, 'was': 7.0, 'sleep': 1.0, '5quot2': 1.0, 'all': 3.0, 'our': 1.0, 'to': 6.0, 'rid': 1.0, 'fussy': 2.0, 'frame': 1.0, 'size': 1.0, 'i': 5.0, 'off': 1.0, 'legs

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [36]:
products = products[products['rating'] != 3]
print(len(products))
products.head()

166752


name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'this':1.0, 'for': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0,'headachesthanks': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1.0, 'for': 1.0,'barnes': 1.0, 'at': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0,'recommend': 1.0, ...",1


Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [37]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products[products['sentiment'] == -1].head()

name,review,rating,word_count,sentiment
Nature's Lullabies SecondYear Sticker Calendar ...,I only purchased asecond-year calendar for ...,2.0,"{'purchasers': 1.0,'future': 1.0, 'other': ...",-1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'kenzie': 1.0, 'my':1.0, 'down': 1.0, 'gi ...",-1
"SoftPlay Cloth Book, Love",This book is boring.Nothing to stimulate my ...,1.0,"{'i': 1.0, 'recommend':1.0, 'of': 1.0, 'waste': ...",-1
Hunnt&reg; FallingFlowers and Birds Kids ...,The reason:Small sizeHardto apply on the wall ...,1.0,"{'roll': 1.0, 'floor':1.0, 'fell': 1.0, 'all': ...",-1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
Cloth Diaper PinsStainless Steel ...,These were good quality--worked fine--heavy ...,2.0,"{'others': 1.0, 'my':1.0, 'filled': 1.0, ...",-1
Cloth Diaper PinsStainless Steel ...,"While the diaper pins areattractive, the metal in ...",2.0,"{'little': 1.0, 'so':1.0, 'baby': 1.0, 'no': ...",-1
Cloth Diaper PinsStainless Steel ...,"The steel part is notstrong at all, unlike ...",1.0,"{'recommend': 1.0,'would': 1.0, 'it': 1.0, ...",-1
Cloth Diaper PinsStainless Steel ...,I really thought I wasgetting a dozen ...,2.0,"{'back': 1.0, 'will':1.0, 'so': 1.0, ...",-1
Super Mario Game NintendoWall Sticker and Decal ...,These do not stick to thewall. They start to peel ...,1.0,"{'them': 1.0, 'you': 1.0,'soon': 1.0, 'as': 2.0, ...",-1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [22]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [38]:
sentiment_model = turicreate.logistic_classifier.create(train_data,
                                                        target = 'sentiment',
                                                        features=['word_count'],
                                                        validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.778177  | 1.930456     | 0.942638          |

| 3         | 12       | 2.778177  | 2.199782     | 0.967822          |

| 4         | 13       | 2.778177  | 2.485848     | 0.976495          |

| 5         | 14       | 2.778177  | 2.753545     | 0.976495          |

+-----------+----------+-----------+--------------+-------------------+

In [61]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 5
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 2.1568

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[offsi]              : 21.7657
word_count[kidsgood]           : 21.6818
word_count[articletest]        : 21.6818
word_count[conclusions]        : 21.6818
word_count[regulatetest]       : 21.6818

Lowest Negative Coefficients
----------------------------
word_count[l8]    

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [65]:
weights = sentiment_model.coefficients
print(weights.column_names())
weights.shape

['name', 'index', 'class', 'value', 'stderr']


(121713, 5)

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [66]:
num_positive_weights = len(weights[weights['value'] >= 0])
num_negative_weights = len(weights[weights['value'] < 0])

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [67]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1.0, 'book':1.0, 'same': 1.0, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1.0, 'waste': 1.0,'wouldnt': 1.0, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [68]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [69]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [70]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[4.788907309214097, -3.000782222462787, -8.18850136076277]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [72]:
y_hat = []
for score in scores:
    if score > 0:
        y_hat.append(1)
    else:
        y_hat.append(-1)
        
y_hat

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [73]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to Turi Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [74]:
def sigmoid_function(val):
    return 1/(1+np.exp(-val))

probs = []
for score in scores:
    probs.append(sigmoid_function(score))
    
probs

[0.9917471313286892, 0.047390547487109365, 0.0002777527712172607]

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [75]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to Turi Create:
[0.9917471313286892, 0.04739054748710937, 0.0002777527712172608]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [141]:
test_data = test_data[['name', 'review', 'rating', 'word_count', 'sentiment']]
s_array = sentiment_model.predict(test_data, output_type='probability')
test_data['score'] = s_array
print(test_data.topk('score', 20))


+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Britax Decathlon Convertib... | I researched a few differe... |  4.0   |
| Lilly Gold Sit 'n' Stroll ... | I just completed a two-mon... |  5.0   |
| Fisher-Price Rainforest Me... | My daughter wasn't able to... |  5.0   |
| Peg Perego Aria Light Weig... | We have 3 strollers...one ... |  5.0   |
| P'Kolino Silly Soft Seatin... | I've purchased both the P'... |  4.0   |
| Evenflo X Sport Plus Conve... | After seeing this in Paren... |  5.0   |
| Britax Frontier Booster Ca... | My family previously owned... |  5.0   |
| Baby Einstein Around The W... | I am so HAPPY I brought th... |  5.0   |
| Infantino Wrap and Tie Bab... | I bought this carrier when... |  5.0   |
| bumGenius One-Size Snap Cl... | Warning: long review!Short... |  5.0   |
+------------------------

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [110]:
test_data.topk('score', 20, reverse=True)

name,review,rating,word_count,sentiment
"Luna Lullaby Bosom BabyNursing Pillow, Choco ...","I have the boppy deluxepillow, the brestfriend ...",5.0,"{'leacho': 1.0, 'bulky':1.0, 'instead': 1.0, ...",1
The First Years TrueChoice P400 Premium ...,Note: we never installedbatteries in these un ...,1.0,"{'features': 1.0, 'nice':1.0, 'seduced': 1.0, ...",-1
Jolly Jumper Arctic SneakA Peek Infant Car Seat ...,"I am a ""research-aholic""in general and have ...",5.0,"{'instead': 1.0,'worthless': 1.0, ...",1
Motorola MBP36 RemoteWireless Video Baby ...,I could go on and onabout the awful ...,4.0,"{'now': 1.0, 'right':1.0, 'time': 1.0, '4': ...",1
VTech Communications Safe&amp; Sounds Full Color ...,"This is my second videomonitoring system, the ...",1.0,"{'beware': 1.0,'thisbuyer': 1.0, 'li ...",-1
Fisher-Price OceanWonders Aquarium Bouncer ...,We have not had ANY luckwith Fisher-Price ...,2.0,"{'taken': 1.0, 'same':1.0, 'mentioned': 1.0, ...",-1
Levana Safe N'See DigitalVideo Baby Monitor with ...,This is the first reviewI have ever written out ...,1.0,"{'from': 1.0, 'anything':1.0, 'purchase': 1.0, ...",-1
Safety 1st High-DefDigital Monitor ...,We bought this babymonitor to replace a ...,1.0,"{'looking': 1.0, 'id':1.0, 'start': 1.0, ...",-1
Snuza Portable BabyMovement Monitor ...,I would have given theproduct 4 stars for w ...,1.0,"{'company': 1.0, 'food':1.0, 'misused': 1.0, ...",-1
Adiri BPA Free NaturalNurser Ultimate Bottle ...,I will try to write anobjective review of the ...,2.0,"{'luck': 1.0,'especially': 1.0, ...",-1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [111]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    data['margin'] = model.predict(data, output_type='margin')
    data['prediction'] = data.apply(lambda x: 1 if x['margin'] > 0 else -1)
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct = len(data[data['sentiment'] == data['prediction']])
    total = len(data)
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct/total
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [114]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [115]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [116]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [117]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [118]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [119]:
print(train_data[0]['word_count'])

{'recommend': 1.0, 'moist': 1.0, 'osocozy': 1.0, 'keps': 1.0, 'leak': 1.0, 'holder': 1.0, 'was': 1.0, 'now': 1.0, 'wipe': 1.0, 'my': 2.0, 'disappointed': 1.0, 'wise': 1.0, 'and': 3.0, 'planet': 1.0, 'it': 3.0, 'love': 1.0, 'early': 1.0, 'not': 2.0, 'highly': 1.0, 'wipes': 1.0, 'bags': 1.0, 'came': 1.0, 'does': 1.0, 'i': 1.0}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [120]:
print(train_data[0]['word_count_subset'])

{'disappointed': 1.0, 'love': 1.0}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [121]:
simple_model = turicreate.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count_subset'],
                                                     validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.416687     | 0.866815          |

| 6         | 7        | 0.492580     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.5013

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [122]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [123]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552027043,0.01208885413305326
word_count_subset,disappointed,1,-2.3550925006107253,0.05041498885569791
word_count_subset,love,1,1.3654354936790372,0.03035462951090517
word_count_subset,little,1,0.5206286360250184,0.021469147566490373
word_count_subset,loves,1,1.6772714555592918,0.04823282753835011
word_count_subset,product,1,-0.320555492995575,0.015431132136201639
word_count_subset,well,1,0.5042567463979283,0.021381300630990037
word_count_subset,great,1,0.9446912694798443,0.020950992659050018
word_count_subset,easy,1,1.1936618983284648,0.029288869202029586
word_count_subset,work,1,-0.6217000124253143,0.02303305979458482


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [124]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |  1.6772714555592918  |
| word_count_subset |   perfect    |   1   |  1.5144862670271348  |
| word_count_subset |     love     |   1   |  1.3654354936790372  |
|    (intercept)    |     None     |   1   |  1.2995449552027043  |
| word_count_subset |     easy     |   1   |  1.1936618983284648  |
| word_count_subset |    great     |   1   |  0.9446912694798443  |
| word_count_subset |    little    |   1   |  0.5206286360250184  |
| word_count_subset |     well     |   1   |  0.5042567463979283  |
| word_count_subset |     able     |   1   | 0.19143830229475087  |
| word_count_subset |     old      |   1   | 0.08539618866781587  |
| word_count_subset |     car      |   1   |  0.0588349900680204  |
| word_count_subset |     less     |   1   | -0.

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [ ]:
# 11

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [126]:
sentiment_model.coefficients

name,index,class,value,stderr
(intercept),None,1,0.779736281462835,None
word_count,recommend,1,0.30524577375829626,None
word_count,moist,1,0.21481280120782906,None
word_count,osocozy,1,0.426802954508828,None
word_count,keps,1,4.519793963568125,None
word_count,leak,1,-0.17272689753089415,None
word_count,holder,1,0.03759775833366718,None
word_count,was,1,-0.029584849955190555,None
word_count,now,1,0.06408793093216988,None
word_count,wipe,1,0.18771870810140973,None


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [131]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.976494573364514

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [132]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [134]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [135]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [137]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)
num_positive/(num_positive+num_negative)

112164
21252


0.8407087605684476

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [138]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)
num_positive/(num_positive+num_negative)

28095
5241


0.8427825773938085

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?